In [1]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
from confluent_kafka.error import KafkaException

In [2]:
def create_topic(topic_name, num_partitions, replication_factor):
    admin_client = AdminClient({'bootstrap.servers': 'kafka-1:29092,kafka-2:29093,kafka-3:29094'})
    
    new_topic = NewTopic(
        topic_name,
        num_partitions=num_partitions,
        replication_factor=replication_factor
    )
    fs = admin_client.create_topics([new_topic])
    
    for topic, f in fs.items():
        try:
            f.result()
            print(f"Topic {topic} created with {num_partitions} partitions and replication factor {replication_factor}")
        except KafkaException as e:
            print(f"Failed to create topic {topic}: {e}")

In [3]:
def manage_partitions(topic_name, num_partitions):
    admin_client = AdminClient({'bootstrap.servers': 'kafka-1:29092,kafka-2:29093,kafka-3:29094'})
    
    existing_topics = admin_client.list_topics(timeout=10).topics
    
    if topic_name not in existing_topics:
        print(f"Topic {topic_name} does not exist. Cannot manage partitions.")
        return
    
    current_partitions = len(existing_topics[topic_name].partitions)
    
    if num_partitions > current_partitions:
        new_partitions = NewPartitions(topic_name, num_partitions)
        fs = admin_client.create_partitions([new_partitions])
        
        for topic, f in fs.items():
            try:
                f.result()
                print(f"Updated {topic} to {num_partitions} partitions")
            except KafkaException as e:
                print(f"Failed to update partitions for topic {topic}: {e}")
    elif num_partitions < current_partitions:
        print(f"Cannot decrease partitions. Current: {current_partitions}, Requested: {num_partitions}")
    else:
        print(f"No change in partitions. Current: {current_partitions}")

    final_partitions = len(admin_client.list_topics(topic=topic_name).topics[topic_name].partitions)
    print(f"Topic {topic_name} final partition count: {final_partitions}")

In [4]:
create_topic('purchase_event', 6,2)

Topic purchase_event created with 6 partitions and replication factor 2
